In [1]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from copy import deepcopy
from collections import deque

from extrapolation import *
from experiments import difference_matrix

In [2]:
torch.cuda.is_available()

True

In [3]:
train_ds = datasets.MNIST("../MNIST", download=True, train=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST("../MNIST", download=True, train=False, transform=transforms.ToTensor())
valid_size = int(0.2 * len(train_ds))
train_ds, valid_ds = data.random_split(train_ds, [len(train_ds) - valid_size, valid_size])

train_loader = data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
valid_loader = data.DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

In [4]:
model = nn.Sequential(
    nn.Conv2d(1, 32, 3),
    nn.ReLU(),
    nn.Conv2d(32, 32, 3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3),
    nn.ReLU(),
    nn.Conv2d(64, 64, 3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(4*4*64, 128),
    nn.ReLU(),
    nn.Linear(128, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [5]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y)
            preds = out.argmax(1)
            ok += (y == preds).sum()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch(loss_log):
    model.train()
    for x, y in tqdm(train_loader):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        loss_log.append(loss.item())
        optimizer.step()

In [6]:
validation(model, valid_loader)

(tensor(0.0660, device='cuda:0'), tensor(2.3024, device='cuda:0'))

## Without acceleration

In [7]:
initial_state = deepcopy(model.state_dict())
optimizer = torch.optim.SGD(model.parameters(), 3e-3)
loss_fn = nn.NLLLoss()

In [8]:
epochs = 30
without_acc = {"train_loss": [], "val_loss": [], "val_acc": []}

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch(without_acc["train_loss"])
    val_acc, val_loss = validation(model, valid_loader)
    without_acc["val_loss"].append(val_loss)
    without_acc["val_acc"].append(val_acc)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Validation accuracy: 0.1144, validation loss: 2.3003
Epoch 2



Validation accuracy: 0.1144, validation loss: 2.2979
Epoch 3



Validation accuracy: 0.1144, validation loss: 2.2926
Epoch 4



Validation accuracy: 0.3968, validation loss: 2.2704
Epoch 5


...


Validation accuracy: 0.9785, validation loss: 0.0689
Epoch 27



Validation accuracy: 0.9806, validation loss: 0.0652
Epoch 28



Validation accuracy: 0.9815, validation loss: 0.0637
Epoch 29



Validation accuracy: 0.9803, validation loss: 0.0682
Epoch 30



Validation accuracy: 0.9812, validation loss: 0.0621


In [9]:
final_without_acc = deepcopy(model.state_dict())

## With acceleration, online scheme

In [10]:
def params_to_vector(parameters):
    param_vectors = []
    for param in parameters:
        param_vectors.append(param.data.flatten().cpu())
    return torch.hstack(param_vectors)

def params_from_vector(parameters, x):
    idx = 0
    for param in parameters:
        n = param.data.numel()
        param.data[:] = x[idx:idx+n].view(param.data.shape)
        idx += n

In [11]:
model.load_state_dict(initial_state)
optimizer = torch.optim.SGD(model.parameters(), 3e-3)

In [12]:
epochs = 30
k = 5
with_acc = {"train_loss": [], "val_loss": [], "val_acc": []}
model_hist = deque(maxlen=k)

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch(with_acc["train_loss"])

    x = params_to_vector(model.parameters())
    if len(model_hist) >= k:
        U = difference_matrix(list(model_hist) + [x])
        x = regularized_RRE(torch.vstack(list(model_hist)), U, 1e-5)
        params_from_vector(model.parameters(), x)
    model_hist.append(x)
    
    val_acc, val_loss = validation(model, valid_loader)
    with_acc["val_loss"].append(val_loss)
    with_acc["val_acc"].append(val_acc)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Validation accuracy: 0.1144, validation loss: 2.3003
Epoch 2



Validation accuracy: 0.1144, validation loss: 2.2979
Epoch 3



Validation accuracy: 0.1144, validation loss: 2.2926
Epoch 4



Validation accuracy: 0.4131, validation loss: 2.2701
Epoch 5


...


Validation accuracy: 0.5152, validation loss: 2.2451
Epoch 27



Validation accuracy: 0.5152, validation loss: 2.2451
Epoch 28



Validation accuracy: 0.5152, validation loss: 2.2451
Epoch 29



Validation accuracy: 0.5152, validation loss: 2.2451
Epoch 30



Validation accuracy: 0.5152, validation loss: 2.2451


## With acceleration, offline scheme

In [15]:
model.load_state_dict(initial_state)
optimizer = torch.optim.SGD(model.parameters(), 3e-3)

In [16]:
epochs = 30
k = 5
with_acc_offline = {"train_loss": [], "val_loss": [], "val_acc": []}
model_hist = deque(maxlen=k)

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch(with_acc_offline["train_loss"])
    
    model_acc = None
    x = params_to_vector(model.parameters())
    if len(model_hist) >= k:
        U = difference_matrix(list(model_hist) + [x])
        y = regularized_RRE(torch.vstack(list(model_hist)), U, 1e-5)
        model_acc = deepcopy(model)
        params_from_vector(model_acc.parameters(), y)
    model_hist.append(x)
        
    if model_acc is None:
        val_acc, val_loss = validation(model, valid_loader)
    else:
        val_acc, val_loss = validation(model_acc, valid_loader)
    with_acc_offline["val_loss"].append(val_loss)
    with_acc_offline["val_acc"].append(val_acc)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Validation accuracy: 0.1144, validation loss: 2.3003
Epoch 2



Validation accuracy: 0.1144, validation loss: 2.2979
Epoch 3



Validation accuracy: 0.1144, validation loss: 2.2926
Epoch 4



Validation accuracy: 0.4125, validation loss: 2.2704
Epoch 5


...


Validation accuracy: 0.9795, validation loss: 0.0657
Epoch 27



Validation accuracy: 0.9799, validation loss: 0.0645
Epoch 28



Validation accuracy: 0.9807, validation loss: 0.0637
Epoch 29



Validation accuracy: 0.9816, validation loss: 0.0627
Epoch 30



Validation accuracy: 0.9812, validation loss: 0.0618


In [17]:
results = {
    "without_acc": without_acc,
    "with_acc_online": with_acc,
    "with_acc_offline": without_acc,
}
torch.save(results, "first_nn_results.p")